In [1]:
import pandas as pd
import numpy as np
import pprint
import re

from bs4 import BeautifulSoup

import json

pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1500)
pd.set_option('display.max_colwidth', 100)

In [2]:
import xml
def remove_tags(text):
    ''.join(xml.etree.ElementTree.fromstring(text).itertext())

In [3]:
fname='/Users/aeevr/Documents/CodeForSanJose/heartofthevalley/artwork-data/consolidated-artworks.json'

# Reading each nest of the json into DF one at a time and saving back to json

In [4]:
#read consolidated-artworks.json into dataframe
#Remember: you had to replace typo herf with href
df = pd.read_json(fname)

In [5]:
#notice type is copied for each record but only one line in original json
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270 entries, 0 to 269
Data columns (total 2 columns):
features    270 non-null object
type        270 non-null object
dtypes: object(2)
memory usage: 6.3+ KB


In [6]:
#features_df is the DF that holds 'features'
features_df=pd.read_json((df['features']).to_json(orient='records'))
features_df.head()

,addToMap,errors,geometry,id,properties,type
0,NaN,NaN,"{'type': 'Point', 'coordinates': [-121.88887, 37.32718]}",10667523,"{'title': 'San José', 'artist': '<a href='https://www.instagram.com/j.duh_/' target='_blank'>J.D...",Feature
1,NaN,NaN,"{'type': 'Point', 'coordinates': [-121.97798, 37.30554]}",23925665,"{'title': 'Nothing Is Too Good for the Working Class', 'artist': 'Carlos Rodriguez', 'descriptio...",Feature
2,NaN,NaN,"{'type': 'Point', 'coordinates': [-121.88582, 37.33022]}",31154388,"{'title': 'Guardian Tiger', 'artist': '<a href='http://mikeborja.tumblr.com/' target='_blank'>Mi...",Feature
3,NaN,NaN,"{'type': 'Point', 'coordinates': [-121.89577, 37.35107]}",38393353,"{'title': 'Kokeshis de San Jo', 'artist': '<a href='http://samrodriguezart.com/' target='_blank'...",Feature
4,NaN,NaN,"{'type': 'Point', 'coordinates': [-121.84637, 37.34036]}",45973257,"{'title': 'A Monument to the Last Barn', 'artist': 'Gloria Bornstein', 'description': 'A native ...",Feature


In [7]:
features_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270 entries, 0 to 269
Data columns (total 6 columns):
addToMap      1 non-null float64
errors        4 non-null object
geometry      270 non-null object
id            270 non-null int64
properties    270 non-null object
type          270 non-null object
dtypes: float64(1), int64(1), object(4)
memory usage: 14.8+ KB


In [8]:
#covert features_df['properties'] into a json and open into its own df==>properties_df
properties_df=pd.read_json((features_df['properties']).to_json(orient='records'))
properties_df.head()

,address,artist,city,country,crossStreet,description,image,postalCode,sourceOfInformation,sourceURL,state,title,website
0,"488 Almaden Blvd, Ste 1800",<a href='https://www.instagram.com/j.duh_/' target='_blank'>J.Duh</a>,San José,United States,NaN,"This mural was painted by <a href='https://www.instagram.com/j.duh_/' target='_blank'>J.Duh</a>,...",NaN,95113,NaN,NaN,CA,San José,NaN
1,1080 Saratoga Ave #15,Carlos Rodriguez,San José,United States,NaN,This mural was painted by artist <a href='https://www.carlosrodriguezart.com/' target='_blank'>C...,NaN,95129,NaN,NaN,CA,Nothing Is Too Good for the Working Class,NaN
2,396 S 1st St.,<a href='http://mikeborja.tumblr.com/' target='_blank'>Mike Borja</a>,San José,United States,NaN,"This mural is the middle in a series of three murals on the exterior wall of The Studio gym, fac...",NaN,95113,NaN,NaN,CA,Guardian Tiger,http://mikeborja.tumblr.com/
3,200 E Taylor St,<a href='http://samrodriguezart.com/' target='_blank'>Sam Rodriguez</a>,San José,United States,NaN,This artbox was painted by <a href='http://samrodriguezart.com/' target='_blank'>Sam Rodriguez</...,NaN,95112,NaN,NaN,CA,Kokeshis de San Jo,NaN
4,647 S King Rd,Gloria Bornstein,San José,United States,NaN,"A native of San José, Emma Prusch bequeathed her 47 acre dairy farm to the City in 1962. The dee...",NaN,95116,NaN,NaN,CA,A Monument to the Last Barn,NaN


# ADD code for fixing data HERE!!

In [9]:
#get new artist names


soup = BeautifulSoup(properties_df['artist'].iloc[0],'lxml')

tmp_col = properties_df['artist']
artist_name = []
artist_site = []
artist_leftover =[]

for i in range(len(tmp_col)):
    #artist_leftover.append(remove_tags(str(tmp_col.loc[i])))
    soup = BeautifulSoup(tmp_col.loc[i], 'lxml')
    print(i,soup)
    
    
    artist_leftover.append(soup.get_text())
    
    
    
    #print(soup)
    tags = soup.find_all('a')
    #print(tags)
    #print(len(tags))
    
    if (len(tags)==1): 
        tags = soup.find('a')
        a_n = tags.text
        a_s = tags.attrs['href']
    elif (len(tags)>1):
        multi_artist_dict = {}
        a_n = []
        a_s =[]
        for t in tags:
            a_n.append(t.text)
            a_s.append(t.attrs['href'])
            print('i=', i,' has more than 1 tag so look closer, tag = ',t)
            print(t.text, t.attrs['href'])
        print('\n')
        
    elif (len(tags)==0):
        a_n = tmp_col[i]
        a_s = ''
    #print('a_n:',a_n,' a_s:',a_s,'\n\n')
    #if (len(a_n.split(' and ')) >1): a_n = [i for i in a_n.split(' and ')];
    
    
    artist_name.append(a_n)
    artist_site.append(a_s)
        
artist_df = pd.concat([properties_df['artist'],
                        pd.DataFrame(artist_leftover,columns=['artist_no_tags']),
                       pd.DataFrame(artist_name,columns=['artist_name']),
                       pd.DataFrame(artist_site,columns=['artist_site'])],axis=1)

#fixed by hand "artist": "Gerald Heffernon", herf==>href, "artist": ["Po Shu Wang", "Louise Bertelsen"],
#

0 <html><body><a href="https://www.instagram.com/j.duh_/" target="_blank">J.Duh</a></body></html>
1 <html><body><p>Carlos Rodriguez</p></body></html>
2 <html><body><a href="http://mikeborja.tumblr.com/" target="_blank">Mike Borja</a></body></html>
3 <html><body><a href="http://samrodriguezart.com/" target="_blank">Sam Rodriguez</a></body></html>
4 <html><body><p>Gloria Bornstein</p></body></html>
5 <html><body><p>Angela Buenning Filo</p></body></html>
6 <html><body><p>Jun Kaneko</p></body></html>
7 <html><body><a href="http://www.andradehemming.com/" target="_blank">Jeffrey Andrade Hemming</a></body></html>
8 <html><body><a href="http://www.gemellosmurals.com/" target="_blank">Lila Gemellos</a></body></html>
9 <html><body><a href="http://www.aarondelacruz.com/" target="_blank">Aaron De La Cruz</a></body></html>
10 <html><body><a href="https://www.instagram.com/aarondelacruz/" target="_blank">Aaron De La Cruz</a></body></html>
11 <html><body><a href="https://www.instagram.com/fco1980/" 

104 <html><body><p>Gerald Heffernon</p></body></html>
105 <html><body><p>Andrew Schoultz</p></body></html>
106 <html><body><a href="http://www.sm-artgroup.com/" target="_blank">Scape Martinez</a></body></html>
107 <html><body><a href="http://samrodriguezart.com/" target="_blank">Sam Rodriguez</a></body></html>
108 <html><body><a href="http://instagram.com/gigamelted/" target="_blank">Amine Rastgar</a>, <a href="http://instagram.com/chgyyrscrenammme/" target="_blank">Robbie Api</a>, <a href="http://instagram.com/griffinone/" target="_blank">Griffin</a>, Doper, and <a href="https://www.instagram.com/wispertfl/" target="_blank">Wisper</a>. <br/> Source: POW! WOW! San José, 2017 Mural &amp; Event Locations https://www.powwowsanjose.com/2017-locations/</body></html>
i= 108  has more than 1 tag so look closer, tag =  <a href="http://instagram.com/gigamelted/" target="_blank">Amine Rastgar</a>
Amine Rastgar http://instagram.com/gigamelted/
i= 108  has more than 1 tag so look closer, tag =  <a

KeyError: 'href'

# TYPO! in record 142 herf should be href

In [10]:
properties_df['artist'].iloc[142]

"<a herf='https://deniskorkh.tumblr.com/' target='_blank'>Denis Korkh</a>"

In [11]:
properties_df['artist'].iloc[142].replace('herf','href')

"<a href='https://deniskorkh.tumblr.com/' target='_blank'>Denis Korkh</a>"

In [12]:
properties_df.ix[142,'artist'] = properties_df['artist'].iloc[142].replace('herf','href')
properties_df['artist'].iloc[142]

/Users/aeevr/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


"<a href='https://deniskorkh.tumblr.com/' target='_blank'>Denis Korkh</a>"

# try to run artist name fixer code again

In [34]:
#get new artist names


#regex expression to split artists names esp. when multiple artists
artist_sep_pattern = re.compile(" and|\+|,|&|, |\u00B7")
#[^\u0000-\u007F][^\u0000-\u00FF]

tmp_col = properties_df['artist']
artist_name = []
artist_site = []
artist_leftover =[]

#loop over each entry in artist col
for i in range(len(tmp_col)):
    #parse entry in BS object
    soup = BeautifulSoup(tmp_col.loc[i], 'lxml')
    
    #the text without tags for QA purposes
    artist_leftover.append(soup.get_text())

    #after testing this is the text I want to use for artist_name
    a_n = soup.get_text()
    
    #If the artist_name has more than 4 'words' print because it is likely to be problematic
    if (len(a_n.split())>5):
        print('i=', i,' has more than 5 words so look closer, text = ',a_n,'\n')

    #check the artist name to see if more than one person, split the string and create a list
    if (len(re.split(artist_sep_pattern, a_n)) >1): a_n = [i for i in re.split(artist_sep_pattern, a_n) if len(i)>0 ];

    #gets html tags with 'a' which is all is in this column
    tags = soup.find_all('a')
    
    #only 1 tag in entry, send the tag's attr to the artist_site column
    if (len(tags)==1): 
        tags = soup.find('a')
        a_s = tags.attrs['href']
        #a_n = tags.text
        #check the artist name to see if more than one person and create a list
        #if (len(re.split(artist_sep_pattern, a_n)) >1): a_n = [i for i in re.split(artist_sep_pattern, a_n)];
        
    #more than one tag in entry, 
    #create list of websites connected to each tag & send to the artist_site column
    #print out info on these to edit manually if neccessary
    elif (len(tags)>1):
        multi_artist_dict = {}
        #a_n = []
        a_s =[]
        for t in tags:
            #a_n.append(t.text)
            a_s.append(t.attrs['href'])
            print('i=', i,' has more than 1 tag so look closer, tag = ',t)
            print(t.text, t.attrs['href'])
        print('\n')
    
    #if no tags (desired behavior!) the the text should b the artist name
    #elif (len(tags)==0):
        #a_n = tmp_col[i]
        #a_s = ''
        #check the artist name to see if more than one person and create a list
        #if (len(re.split(artist_sep_pattern, a_n))>1): a_n = [i for i in re.split(artist_sep_pattern, a_n)];
            
    #print('a_n:',a_n,' a_s:',a_s,'\n\n')
    
    artist_name.append(a_n)
    artist_site.append(a_s)
        
artist_df = pd.concat([properties_df['artist'],
                        pd.DataFrame(artist_leftover,columns=['artist_no_tags']),
                       pd.DataFrame(artist_name,columns=['artist_name']),
                       pd.DataFrame(artist_site,columns=['artist_site'])],axis=1)


artist_df.head(len(artist_df))

i= 17  has more than 5 words so look closer, text =  Anne Chamberlain and Victor Maria Zaballa 

i= 23  has more than 5 words so look closer, text =  Academy of Art University Graduate Students under the direction of Professor Donna Schumacher 

i= 28  has more than 5 words so look closer, text =  Living Lenses – Po Shu Wang and Louise Bertelsen 

i= 36  has more than 5 words so look closer, text =  Carolyn Ryder Cooley, Lena Wolff, Daniel Jesse Lewis, Jessie Rose Vala, David Choe, Bruno 9Li, Klone, Adrian Lee, and Know Hope 

i= 39  has more than 5 words so look closer, text =  MMAP (Mural Music and Arts Project), lead artist: Omar Ramirez, and students and staff 

i= 45  has more than 5 words so look closer, text =  Merge Conceptual Design (Franka Diehnelt and Claudia Reisenberger) 

i= 47  has more than 5 words so look closer, text =  San Jose State University “Art in Community” Students with Tony May, Instructor 

i= 50  has more than 1 tag so look closer, tag =  <a href="https://w

,artist,artist_no_tags,artist_name,artist_site
0,<a href='https://www.instagram.com/j.duh_/' target='_blank'>J.Duh</a>,J.Duh,J.Duh,https://www.instagram.com/j.duh_/
1,Carlos Rodriguez,Carlos Rodriguez,Carlos Rodriguez,https://www.instagram.com/j.duh_/
2,<a href='http://mikeborja.tumblr.com/' target='_blank'>Mike Borja</a>,Mike Borja,Mike Borja,http://mikeborja.tumblr.com/
3,<a href='http://samrodriguezart.com/' target='_blank'>Sam Rodriguez</a>,Sam Rodriguez,Sam Rodriguez,http://samrodriguezart.com/
4,Gloria Bornstein,Gloria Bornstein,Gloria Bornstein,http://samrodriguezart.com/
5,Angela Buenning Filo,Angela Buenning Filo,Angela Buenning Filo,http://samrodriguezart.com/
6,Jun Kaneko,Jun Kaneko,Jun Kaneko,http://samrodriguezart.com/
7,<a href='http://www.andradehemming.com/' target='_blank'>Jeffrey Andrade Hemming</a>,Jeffrey Andrade Hemming,Jeffrey Andrade Hemming,http://www.andradehemming.com/
8,<a href='http://www.gemellosmurals.com/' target='_blank'>Lila Gemellos</a>,Lila Gemellos,Lila Gemellos,http://www.gemellosmurals.com/
9,<a href='http://www.aarondelacruz.com/' target='_blank'>Aaron De La Cruz</a>,Aaron De La Cruz,Aaron De La Cruz,http://www.aarondelacruz.com/


In [80]:
#create printout of artist names to flag messed up looking entries
artist_df[['artist_no_tags','artist_name']].to_html('/tmp/table.html')

In [17]:
flagged_ind = [
                0,23,28,29,36,
                38,39,45,47,52,53,59,71,
                75,84,101,102,108,
                113,114,119,132,134,136,141,146,
                151,152,154,160,162,163,165,168,170,
                187,196,198,203,207,216,217,219,223,
                224,225,230,236,237,238,242,248,250,255,
                263,265,266,268
              ]
len(flagged_ind)


58

# Fix entry 160 which has all the description text stuck in artist_name

In [45]:
artist_df.iat[160,2]

['Gerald Heffernon This fun',
 ' exuberant\xa0sculpture links the themes of wild animal',
 ' domestic dog. A bronze beaver in the creek slaps the water with its tail',
 ' creates an over-sized splash of stainless steel.\xa0 An extension of the splash sends a bright yellow ball in a high-arcing trajectory ten feet high.\xa0  Fifteen feet away',
 ' a bronze dog leaps ten feet in the air',
 ' in visual time lapse sequence catches the ball',
 ' completing the connection between the two sculptural elements',
 ' the two worlds.\xa0\xa0District6\xa0Report a concern\xa0(408) 793-04330 or\xa0publicart@sanjoseca.go']

In [46]:
artist_df.iat[160,2] = 'Gerald Heffernon'

In [48]:
artist_df.iloc[160,2]

'Gerald Heffernon'

# Moving on to fix description column. 

# Rest of the artist_name's to be fixed via googling

## REMEMBER: artist_name should replace 'artist' col in properties DF 
and artist_site should be attached to properties DF 

In [51]:
[print(artist_df.iloc[108,i]) for i in range(3)]

<a href='http://instagram.com/gigamelted/' target='_blank'>Amine Rastgar</a>, <a href='http://instagram.com/chgyyrscrenammme/' target='_blank'>Robbie Api</a>, <a href='http://instagram.com/griffinone/' target='_blank'>Griffin</a>, Doper, and <a href='https://www.instagram.com/wispertfl/' target='_blank'>Wisper</a>. <br/> Source: POW! WOW! San José, 2017 Mural & Event Locations https://www.powwowsanjose.com/2017-locations/
Amine Rastgar, Robbie Api, Griffin, Doper, and Wisper.  Source: POW! WOW! San José, 2017 Mural & Event Locations https://www.powwowsanjose.com/2017-locations/
['Amine Rastgar', ' Robbie Api', ' Griffin', ' Doper', '', ' Wisper.  Source: POW! WOW! San Jos', '', ' 2017 Mural ', ' Event Locations https://www.powwowsanjose.com/2017-locations/']


[None, None, None]

In [52]:
artist_df[['artist_name','artist_site']].head(len(artist_df))

,artist_name,artist_site
0,J.Duh,https://www.instagram.com/j.duh_/
1,Carlos Rodriguez,
2,Mike Borja,http://mikeborja.tumblr.com/
3,Sam Rodriguez,http://samrodriguezart.com/
4,Gloria Bornstein,
5,Angela Buenning Filo,
6,Jun Kaneko,
7,Jeffrey Andrade Hemming,http://www.andradehemming.com/
8,Lila Gemellos,http://www.gemellosmurals.com/
9,Aaron De La Cruz,http://www.aarondelacruz.com/


In [43]:
artist_df.artist_no_tags.head(len(artist_df))

0                                                                                                    J.Duh
1                                                                                         Carlos Rodriguez
2                                                                                               Mike Borja
3                                                                                            Sam Rodriguez
4                                                                                         Gloria Bornstein
5                                                                                     Angela Buenning Filo
6                                                                                               Jun Kaneko
7                                                                                  Jeffrey Andrade Hemming
8                                                                                            Lila Gemellos
9                                    

In [ ]:
#get new artist websites

In [ ]:
#source column

In [45]:
#change artist name column to simple txt string getting rid of any html tags
properties_df['artist'] = artist_name
properties_df['artist_site'] = artist_site
properties_df.head()

,address,artist,city,country,crossStreet,description,image,postalCode,sourceOfInformation,sourceURL,state,title,website,artist_site
0,"488 Almaden Blvd, Ste 1800",J.Duh,San José,United States,NaN,"This mural was painted by <a href='https://www.instagram.com/j.duh_/' target='_blank'>J.Duh</a>,...",NaN,95113,NaN,NaN,CA,San José,NaN,https://www.instagram.com/j.duh_/
1,1080 Saratoga Ave #15,Carlos Rodriguez,San José,United States,NaN,This mural was painted by artist <a href='https://www.carlosrodriguezart.com/' target='_blank'>C...,NaN,95129,NaN,NaN,CA,Nothing Is Too Good for the Working Class,NaN,
2,396 S 1st St.,Mike Borja,San José,United States,NaN,"This mural is the middle in a series of three murals on the exterior wall of The Studio gym, fac...",NaN,95113,NaN,NaN,CA,Guardian Tiger,http://mikeborja.tumblr.com/,http://mikeborja.tumblr.com/
3,200 E Taylor St,Sam Rodriguez,San José,United States,NaN,This artbox was painted by <a href='http://samrodriguezart.com/' target='_blank'>Sam Rodriguez</...,NaN,95112,NaN,NaN,CA,Kokeshis de San Jo,NaN,http://samrodriguezart.com/
4,647 S King Rd,Gloria Bornstein,San José,United States,NaN,"A native of San José, Emma Prusch bequeathed her 47 acre dairy farm to the City in 1962. The dee...",NaN,95116,NaN,NaN,CA,A Monument to the Last Barn,NaN,


In [46]:
#convert properties_df back to json and load into a dict
properties_json = properties_df.to_json(orient='records')
properties_json_dict = json.loads(properties_json)

#write the properties to json for QA in pretty format
with open('properties.json', 'w') as f:
    pprint.pprint(properties_json_dict, stream=f)
f.close()
pprint.pprint(properties_json_dict)

[{'address': '488 Almaden Blvd, Ste 1800',
  'artist': 'J.Duh',
  'artist_site': 'https://www.instagram.com/j.duh_/',
  'city': 'San José',
  'country': 'United States',
  'crossStreet': None,
  'description': 'This mural was painted by <a '
                 "href='https://www.instagram.com/j.duh_/' "
                 "target='_blank'>J.Duh</a>, and produced in collaboration "
                 "with <a href='https://www.exhibitiondistrict.com/' "
                 "target='_blank'>Exhibition District</a>. It is located on "
                 'the exterior of the PWC Building on Balbach St. <br/> '
                 'Source: Exhibition District, '
                 'https://www.exhibitiondistrict.com/',
  'image': None,
  'postalCode': 95113,
  'sourceOfInformation': None,
  'sourceURL': None,
  'state': 'CA',
  'title': 'San José',
  'website': None},
 {'address': '1080 Saratoga Ave #15',
  'artist': 'Carlos Rodriguez',
  'artist_site': '',
  'city': 'San José',
  'country': 'United States

  'sourceURL': None,
  'state': 'CA',
  'title': 'Reflection Eternal',
  'website': None},
 {'address': 'S 1st St between San Fernando and Post St',
  'artist': 'Phuong-Mai Bui-Quang',
  'artist_site': '',
  'city': 'San José',
  'country': 'United States',
  'crossStreet': None,
  'description': "<a href='https://www.facebook.com/pg/pmbq.studios/' "
                 "target='_blank'>Phuong-Mai Bui-Quang</a> painted this mural "
                 'of a panda cafe and bakery. This mural is located on a wall '
                 'along S 1st Street between San Fernando and Post Streets in '
                 "downtown San José. <br/><br/> <a href='' "
                 "target='_blank'>Learn more</a>.",
  'image': None,
  'postalCode': 95113,
  'sourceOfInformation': None,
  'sourceURL': None,
  'state': 'CA',
  'title': 'Panda Cafe and Bakery',
  'website': None},
 {'address': '899 S 1st St.',
  'artist': 'Tomas Talamantes',
  'artist_site': 'https://www.facebook.com/wisper.t.talamantes',
  

  'website': None},
 {'address': '195 S 28th St.',
  'artist': 'Unknown',
  'artist_site': '',
  'city': 'San José',
  'country': 'United States',
  'crossStreet': 'at S 28th and E San Antonio St.',
  'description': 'Source: Velasquez, Carlos. “POW! WOW! San Jose 2017 Mural '
                 'Ride.“ Ride With GPS, 29, Oct. 2017, '
                 'https://ridewithgps.com/routes/25823991',
  'image': None,
  'postalCode': 95116,
  'sourceOfInformation': None,
  'sourceURL': None,
  'state': 'CA',
  'title': 'Virgin de Guadalupe',
  'website': None},
 {'address': '349 S 1st St',
  'artist': 'Royyal Dog',
  'artist_site': 'https://www.instagram.com/royyaldog/',
  'city': 'San José',
  'country': 'United States',
  'crossStreet': None,
  'description': 'Mural of legendary singer and songwriter Sade. Sade (Helen '
                 'Folasade Adu or Sade Adu) is the lead vocalist of the iconic '
                 'British band Sade, and a British Nigerian singer and '
                 'songw

  'website': None},
 {'address': '1402 Monterey Hwy',
  'artist': 'Carlos Rodriguez',
  'artist_site': '',
  'city': 'San José',
  'country': 'United States',
  'crossStreet': None,
  'description': 'This mural was painted by artist <a '
                 "href='https://www.carlosrodriguezart.com/' "
                 "target='_blank'>Carlos Rodriguez</a>; it is located at DCP "
                 'El Camino Middle School. Source: Carlos Rodriguez, '
                 'https://www.carlosrodriguezart.com/',
  'image': None,
  'postalCode': 95110,
  'sourceOfInformation': None,
  'sourceURL': None,
  'state': 'CA',
  'title': 'Mural at DCP El Camino Middle School',
  'website': None},
 {'address': '6445 Camden Avenue',
  'artist': 'Lisa Reinertson',
  'artist_site': '',
  'city': 'San Jose',
  'country': None,
  'crossStreet': None,
  'description': 'The courtyard artwork is a life size image of a native '
                 'Californian: an Ohlone woman in a tulle reed boat. For the '
        

                 'intended to act as a trellis for the\xa0vine growing on '
                 'portions of the\xa0wall below.',
  'image': 'http://sanjoseca.gov/ImageRepository/Path?filePath=%2fdocuments%5c00000000-0000-0000-0000-000000000000%5c1760%5c2418%5c2729%2fP1110433_201404292333355643.JPG&thumbnailSize=Thumb400X400&preserveAspect=true',
  'postalCode': 95110,
  'sourceOfInformation': 'http://sanjoseca.gov/Facilities',
  'sourceURL': 'http://sanjoseca.gov/Facilities/Facility/Details/Public-Art-Big-Flowers-Adobe-Headquarter-514',
  'state': 'CA',
  'title': 'Big Flowers',
  'website': None},
 {'address': '150 W San Carlos St',
  'artist': 'Soo-in Yang',
  'artist_site': '',
  'city': 'San Jose',
  'country': None,
  'crossStreet': None,
  'description': 'Creating a social space bringing together Convention Center '
                 'event goers and the general public, the project’s central '
                 'element is a 40’ diameter canopy, supported by three groups '
          

...

                 'alphabets as the foundation of the written word, which in '
                 'turn is fundamental to libraries and illumination of the '
                 'mind. Each pane contains characters in different scripts '
                 'that are the basis for written Latin, Russian, Vietnamese '
                 'and some Indian languages. These are lineages of the '
                 'languages that presently are predominant in the Pearl Avenue '
                 'Branch Library collection. The lamp is inscribed with '
                 'ancient Sumerian cuneiform script which repeats the text "We '
                 'are all one." \xa0The lamp is lit with color changing LEDs, '
                 'programmed to cycle on 30-minute intervals through the color '
                 'spectrum. In this way the lamp also functions as a '
                 'timepiece.\xa0Another critical but unseen aspect to the '
                 'artwork are photovoltaic cells that are embedded in the

  'website': None},
 {'address': '194 S. Market St. ',
  'artist': 'Robert Graham',
  'artist_site': '',
  'city': 'San José',
  'country': 'United States',
  'crossStreet': None,
  'description': 'This statue is sculpted by Robert Graham. It is located in '
                 'Plaza de César Chávez. Image: Yan-Yin Choy. <br/><br/> '
                 'Source: City of San José Public Art Program, '
                 'http://www.sanjoseca.gov/Facilities/',
  'image': 'img/Quetzalcoatl_sculptureby_RobertGraham_photoby_YanYinChoy.jpg',
  'postalCode': 95113,
  'sourceOfInformation': None,
  'sourceURL': None,
  'state': 'CA',
  'title': 'Quetzalcoatl',
  'website': None},
 {'address': '260 N 1st St.',
  'artist': 'Galen Oback',
  'artist_site': 'https://www.instagram.com/galenobackart/',
  'city': 'San José',
  'country': 'United States',
  'crossStreet': None,
  'description': 'This mural was painted by <a '
                 "href='https://www.instagram.com/galenobackart/' "
                

  'description': 'In 2009, Wowhause created Wishing Wands. Wishing Wands evoke '
                 'the childhood tradition of making a wish by blowing on a '
                 'dandelion seed head. These interactive sculptures are '
                 'designed to be accessible to children of all physical '
                 'capabilities: each seed head features 38 2 1/2 inch Austrian '
                 'Crystals that catch the light, refracting sunlight in '
                 "prismatic rainbows children can 'catch' and play in. "
                 '<br/><br/> The three sculptures are sited near an accessible '
                 'playground in Berryessa Creek Park. Schoolchildren '
                 'participated in making wishes on each of the crystals before '
                 'they were incorporated into the sculptures. '
                 '<br/><br/>Source: City of San José Public Art Program, '
                 'http://www.sanjoseca.gov/Facilities/Facility/Details/Public-Art-Wishing-Wand

  'artist_site': ['https://www.facebook.com/SerinaSerjama',
                  'https://www.facebook.com/WomenRisingSJ/',
                  'https://www.facebook.com/HeartofChaosArtisanCollective'],
  'city': 'San José',
  'country': 'United States',
  'crossStreet': None,
  'description': 'You can find this mural on the exterior side of Grocery '
                 'Outlet. This mural is painted by <a '
                 "href='https://www.facebook.com/SerinaSerjama' "
                 "target='_blank'>Serina Serjama</a>, in collaboration with <a "
                 "href='https://www.facebook.com/WomenRisingSJ/' "
                 "target='_blank'>Women Rising San José</a> and <a "
                 "href='https://www.facebook.com/HeartofChaosArtisanCollective' "
                 "target='_blank'>Heart of Chaos</a>. Image: Yan-Yin Choy.",
  'image': 'img/7thStMuralProject_muralby_SerinaSerjama_WomenRisingSJ_HeartofChaos_photoby_YanYinChoy.jpg',
  'postalCode': 95113,
  'sourceOfInformation

  'postalCode': 95127,
  'sourceOfInformation': None,
  'sourceURL': None,
  'state': 'CA',
  'title': "Heart of the Valley's Delight",
  'website': None},
 {'address': '1102 E Santa Clara St',
  'artist': 'Paul J. Gonzalez',
  'artist_site': 'http://www.pauljgonzalezartstudios.com/',
  'city': 'San José',
  'country': 'United States',
  'crossStreet': None,
  'description': 'In 1997, Paul J Gonalzez painted murals on columns inside '
                 'the East San José Carnegie Branch Library. View a <a '
                 "href='https://www.facebook.com/photo.php?fbid=10155359372619833&set=a.10155359372279833.1073741848.558994832&type=3&theater' "
                 "target='_blank'>photo of the mural.</a> <br/> Source: Paul "
                 'J. Gonzalez Art Studios, '
                 'http://www.pauljgonzalezartstudios.com/',
  'image': None,
  'postalCode': 95116,
  'sourceOfInformation': None,
  'sourceURL': None,
  'state': 'CA',
  'title': 'Murals on Columns inside East San José

In [57]:
#now get the properties json dict for each row(or record) 
#and replace the properties column in the features df
#with updated properties

a=[]
for record in range(len(properties_json_dict)):
    a.append(properties_json_dict[record])
    
features_df['properties']=a
features_df.loc[45:55,:].head(10)

,addToMap,errors,geometry,id,properties,type
45,NaN,NaN,"{'type': 'Point', 'coordinates': [-121.8014591823, 37.2997791496]}",567684018,"{'address': '1924 Yerba Buena Road', 'artist': 'Merge Conceptual Design (Franka Diehnelt and Cla...",Feature
46,NaN,NaN,"{'type': 'Point', 'coordinates': [-121.872449, 37.344869]}",574209101,"{'address': '901 E Santa Clara St.', 'artist': 'Scape Martinez', 'city': 'San Jose', 'country': ...",Feature
47,NaN,NaN,"{'type': 'Point', 'coordinates': [-121.8809223, 37.313529]}",602095010,"{'address': 'Floyd and Locust St', 'artist': 'San Jose State University “Art in Community” Stude...",Feature
48,NaN,NaN,"{'type': 'Point', 'coordinates': [-121.8116598598, 37.3156456]}",604987115,"{'address': '1865 Monrovia Dr', 'artist': 'Cynthia Handel', 'city': 'San Jose', 'country': None,...",Feature
49,NaN,NaN,"{'type': 'Point', 'coordinates': [-121.9312634, 37.2513184]}",606590275,"{'address': '3369 Union Ave', 'artist': 'Michael Stutz', 'city': 'San Jose', 'country': None, 'c...",Feature
50,NaN,NaN,"{'type': 'Point', 'coordinates': [-121.92456, 37.33404]}",609391426,"{'address': '1701 Park Ave', 'artist': ['Lacey Bryant', 'Ben Henderson'], 'city': 'San José', 'c...",Feature
51,NaN,NaN,"{'type': 'Point', 'coordinates': [-121.87912, 37.32193]}",622984454,"{'address': '980 S 1st Street Ste B', 'artist': ['Jim Fonseca', 'Ivan Gonzalez'], 'city': 'San J...",Feature
52,NaN,NaN,"{'type': 'Point', 'coordinates': [-121.86208, 37.34577]}",646621349,"{'address': '195 S 28th St.', 'artist': 'Unknown', 'city': 'San José', 'country': 'United States...",Feature
53,NaN,NaN,"{'type': 'Point', 'coordinates': [-121.8867007, 37.3306511]}",666599470,"{'address': '349 S 1st St', 'artist': 'Royyal Dog', 'city': 'San José', 'country': 'United State...",Feature
54,NaN,NaN,"{'type': 'Point', 'coordinates': [-121.891267, 37.3471033]}",676826704,"{'address': '488 N. 6 th Street', 'artist': 'Diana Pumpelly-Bates', 'city': 'San Jose', 'country...",Feature


In [58]:
#convert features df into json
features_json = features_df.to_json(orient='records')

#load features json into a dict
features_json_dict = json.loads(features_json)

#write features json dict to a json file in pretty format for QA and for later use
with open('features.json', 'w') as f:
    pprint.pprint(features_json_dict, stream=f)
f.close()
#pprint.pprint(features_json_dict)

In [60]:
pprint.pprint(features_json_dict)

[{'addToMap': None,
  'errors': None,
  'geometry': {'coordinates': [-121.88887, 37.32718], 'type': 'Point'},
  'id': 10667523,
  'properties': {'address': '488 Almaden Blvd, Ste 1800',
                 'artist': 'J.Duh',
                 'artist_site': 'https://www.instagram.com/j.duh_/',
                 'city': 'San José',
                 'country': 'United States',
                 'crossStreet': None,
                 'description': 'This mural was painted by <a '
                                "href='https://www.instagram.com/j.duh_/' "
                                "target='_blank'>J.Duh</a>, and produced in "
                                'collaboration with <a '
                                "href='https://www.exhibitiondistrict.com/' "
                                "target='_blank'>Exhibition District</a>. It "
                                'is located on the exterior of the PWC '
                                'Building on Balbach St. <br/> Source: '
           

  'errors': None,
  'geometry': {'coordinates': [-121.9139669, 37.3522017], 'type': 'Point'},
  'id': 332981421,
  'properties': {'address': '1701 Airport Blvd.',
                 'artist': 'Carlos Perez',
                 'artist_site': '',
                 'city': 'San Jose',
                 'country': None,
                 'crossStreet': None,
                 'description': 'This illustrated wall installation recognizes '
                                'three prime-movers behind San Jose’s airport: '
                                'Ernie Renzel, James Nissen and Norman Y. '
                                'Mineta.',
                 'image': 'http://sanjoseca.gov/ImageRepository/Path?filePath=%2fdocuments%5c00000000-0000-0000-0000-000000000000%5c1760%5c2418%5c2891%2fminetacompfin_201406111912324378.jpg&thumbnailSize=Thumb400X400&preserveAspect=true',
                 'postalCode': 95110,
                 'sourceOfInformation': 'http://sanjoseca.gov/Facilities',
                

                 'title': 'Circles and Stars',
                 'website': None},
  'type': 'Feature'},
 {'addToMap': None,
  'errors': None,
  'geometry': {'coordinates': [-121.9139669, 37.3522017], 'type': 'Point'},
  'id': 507374658,
  'properties': {'address': '1701 Airport Boulevard',
                 'artist': 'Ben Hooker + Shona Kitchen',
                 'artist_site': '',
                 'city': 'San Jose',
                 'country': None,
                 'crossStreet': None,
                 'description': 'Airport information and security systems '
                                'animate a dynamic aquatic ecosystem of '
                                'schooling fish, underwater cameras and video '
                                'screens. This artwork is both a celebration '
                                'of the beauty of complex technological and '
                                'organic systems and a statement about the '
                                'ubiquitous

                                'sculptural ceilings, curious light '
                                'projections, and more traditional sculpture.',
                 'image': 'http://sanjoseca.gov/ImageRepository/Path?filePath=%2fdocuments%5c00000000-0000-0000-0000-000000000000%5c1760%5c2418%5c2655%2fSkeptacle+photo+credit+gregory+case_201404072037229811.jpg&thumbnailSize=Thumb400X400&preserveAspect=true',
                 'postalCode': 95112,
                 'sourceOfInformation': 'http://sanjoseca.gov/Facilities',
                 'sourceURL': 'http://sanjoseca.gov/Facilities/Facility/Details/Public-Art-Recolecciones-MLK-Jr-Library-455',
                 'state': 'CA',
                 'title': 'Recolecciones',
                 'website': None},
  'type': 'Feature'},
 {'addToMap': None,
  'errors': None,
  'geometry': {'coordinates': [-121.8014591823, 37.2997791496],
               'type': 'Point'},
  'id': 567684018,
  'properties': {'address': '1924 Yerba Buena Road',
           

                 'title': 'My Rabbits, Stories in Clay',
                 'website': None},
  'type': 'Feature'},
 {'addToMap': None,
  'errors': None,
  'geometry': {'coordinates': [-121.8866134994, 37.3496427321],
               'type': 'Point'},
  'id': 836312896,
  'properties': {'address': '491 E. Empire St.',
                 'artist': 'Masayuki Nagase',
                 'artist_site': '',
                 'city': 'San Jose',
                 'country': None,
                 'crossStreet': None,
                 'description': 'Mr. Nagase describes his concept of a tree as '
                                'metaphor for the body and soul of the '
                                'earliest inhabitants. Trees encompass an '
                                'iconic role in community life. Throughout '
                                'time trees have served as gathering places, '
                                'and learning takes place beneath their '
                                

...

  'properties': {'address': '1701 Airport Blvd.',
                 'artist': 'Banny Banerjee, Matt Gorbet, Susan LK Gorbet and '
                           'Margaret Orth',
                 'artist_site': '',
                 'city': 'San Jose',
                 'country': None,
                 'crossStreet': None,
                 'description': 'Explore time through the movement of this '
                                'suspended sculpture that breathes, ripples '
                                'and reacts to passengers while maintaining '
                                'its abacus-like counting pattern.\xa0 As each '
                                'row completes its count, the movement is '
                                'marked by the closing of a single '
                                'element.\xa0 The counting then resumes until '
                                'the total array closes; time stands still '
                                'until the counting begins again.'

  'type': 'Feature'},
 {'addToMap': None,
  'errors': None,
  'geometry': {'coordinates': [-121.86002, 37.40904], 'type': 'Point'},
  'id': 3420150700,
  'properties': {'address': 'Isadora Drive and Messina Drive',
                 'artist': 'Wowhause',
                 'artist_site': '',
                 'city': 'San José',
                 'country': 'United States',
                 'crossStreet': None,
                 'description': 'In 2009, Wowhause created Wishing Wands. '
                                'Wishing Wands evoke the childhood tradition '
                                'of making a wish by blowing on a dandelion '
                                'seed head. These interactive sculptures are '
                                'designed to be accessible to children of all '
                                'physical capabilities: each seed head '
                                'features 38 2 1/2 inch Austrian Crystals that '
                                'catch the l

  'id': 3726201520,
  'properties': {'address': '52 E Taylor St',
                 'artist': 'Abel Gonzalez',
                 'artist_site': '',
                 'city': 'San José',
                 'country': 'United States',
                 'crossStreet': None,
                 'description': 'This mural is painted by Abel Gonzalez, and '
                                'produced in collaboration with <a '
                                "href='https://www.empiresevenstudios.com' "
                                "target='_blank'>Empire Seven Studios</a>. "
                                '<br/><br/> Source: <p>Empire Seven Studios, '
                                'Rasteroids, and Genevieve Santos. “Japantown '
                                'Artwalk Map.“ Japantown Artwalk, 2017, '
                                'http://jtownartwalk.com/wp-content/uploads/2016/06/jtown-artwalk-map-1.pdf',
                 'image': 'https://static1.squarespace.com/static/5681ee28841abae3b7d8dec

                                'NOAA weather data to compose different '
                                'patterns of light on the ceiling. (e.g.: '
                                '0-5mph winds vs thunderstorms)',
                 'image': 'http://sanjoseca.gov/ImageRepository/Path?filePath=%2fdocuments%5c00000000-0000-0000-0000-000000000000%5c1760%5c2418%5c6736%2fsensing+you+Adrian.jpg&thumbnailSize=Thumb400X400&preserveAspect=true',
                 'postalCode': 95113,
                 'sourceOfInformation': 'http://sanjoseca.gov/Facilities',
                 'sourceURL': 'http://sanjoseca.gov/Facilities/Facility/Details/Public-Art-Sensing-YOU-Sensing-WATER-Hwy-555',
                 'state': 'CA',
                 'title': 'Sensing YOU / Sensing WATER',
                 'website': None},
  'type': 'Feature'},
 {'addToMap': None,
  'errors': None,
  'geometry': {'coordinates': [-121.83304, 37.2841], 'type': 'Point'},
  'id': 4074804993,
  'properties': {'address': '3590 Cas Dr',
 

                                'three-foot\xa0squares of neon light '
                                'controlled by individual random fading '
                                'transformers. The rectangular mass created\xa0'
                                'by the 24 boxes illuminates the building '
                                'façade. The squares of light float, change '
                                'color, and fade\xa0constantly, but subtly, '
                                'create new patterns and compositions.',
                 'image': 'http://sanjoseca.gov/ImageRepository/Path?filePath=%2fdocuments%5c00000000-0000-0000-0000-000000000000%5c1760%5c2418%5c2748%2f4th++SF+Garage+SF+facade+compressed+photo+credit+gregory+case_201404301606284310.jpg&thumbnailSize=Thumb400X400&preserveAspect=true',
                 'postalCode': 95113,
                 'sourceOfInformation': 'http://sanjoseca.gov/Facilities',
                 'sourceURL': 'http://sanjoseca.gov/Facilities/Facili

# don't do this b/c the resultant df doesn't give back original json format

#now get the features json dict for each row(or record) 
#and replace the features column in df
#with updated features

a=[]
for record in range(len(features_json_dict)):
    a.append(features_json_dict[record])
    
df['features']=a
df.head()

# Load original json into a dict and replace features data with features_json_dict

In [59]:
with open(fname) as f:
    original_dict = json.load(f)
f.close()

original_dict['features'] = features_json_dict

with open('consolidated-artworks-fixed.json', 'w') as f:
    pprint.pprint(original_dict, stream=f)
f.close()

#used original_dict.keys()

In [64]:
pprint.pprint(original_dict)

{'features': [{'addToMap': None,
               'errors': None,
               'geometry': {'coordinates': [-121.88887, 37.32718],
                            'type': 'Point'},
               'id': 10667523,
               'properties': {'address': '488 Almaden Blvd, Ste 1800',
                              'artist': 'J.Duh',
                              'artist_site': 'https://www.instagram.com/j.duh_/',
                              'city': 'San José',
                              'country': 'United States',
                              'crossStreet': None,
                              'description': 'This mural was painted by <a '
                                             "href='https://www.instagram.com/j.duh_/' "
                                             "target='_blank'>J.Duh</a>, and "
                                             'produced in collaboration with '
                                             '<a '
                                             "href='http

                                             'changes based on environment. '
                                             'The public will observe this '
                                             'relationship between scale and '
                                             'environment when standing next '
                                             'to these grand seven foot tall '
                                             'sculptures.',
                              'image': 'http://sanjoseca.gov/ImageRepository/Path?filePath=%2fdocuments%5c00000000-0000-0000-0000-000000000000%5c1760%5c2418%5c2742%2fKaneko_63_201404301344087749.jpg&thumbnailSize=Thumb400X400&preserveAspect=true',
                              'postalCode': 95113,
                              'sourceOfInformation': 'http://sanjoseca.gov/Facilities',
                              'sourceURL': 'http://sanjoseca.gov/Facilities/Facility/Details/Public-Art-Convergence-San-Jose-Repertor-418',
                     

                                             'was often said that she could '
                                             '“teach a rock to read.” \xa0'
                                             'Teaching Stones consists of two '
                                             'primary elements. The first is a '
                                             'reading/sitting area, inspired '
                                             'by one of Doris’ favorite '
                                             'activities: story telling. The '
                                             'reading area provides a place to '
                                             'sit and share stories in the '
                                             'traditional manner of gathering '
                                             'in a circle. The decomposed '
                                             'granite interior of the circle '
                                             'extends outwar

               'properties': {'address': '2031 Pulgas Avenue',
                              'artist': 'MMAP (Mural Music and Arts Project)',
                              'artist_site': 'http://www.muralmusicarts.org/',
                              'city': 'East Palo Alto',
                              'country': 'United States',
                              'crossStreet': None,
                              'description': 'In 2001, <a '
                                             "href='http://www.muralmusicarts.org/' "
                                             "target='_blank'>MMAP (Mural "
                                             'Music and Arts Project)</a> and '
                                             'lead artist Omar Ramirez '
                                             'collaborated with students and '
                                             'staff at Boys & Girls Clubs of '
                                             'the Peninsula - Moldaw-Zaffaroni '


                                             'dances.',
                              'image': 'http://sanjoseca.gov/ImageRepository/Path?filePath=%2fdocuments%5c00000000-0000-0000-0000-000000000000%5c1760%5c2418%5c2878%2fMvc-002l_201406111803009248.jpg&thumbnailSize=Thumb400X400&preserveAspect=true',
                              'postalCode': 95112,
                              'sourceOfInformation': 'http://sanjoseca.gov/Facilities',
                              'sourceURL': 'http://sanjoseca.gov/Facilities/Facility/Details/Public-Art-Transcendance-Northside-Commu-453',
                              'state': 'CA',
                              'title': 'Transcendance',
                              'website': None},
               'type': 'Feature'},
              {'addToMap': None,
               'errors': None,
               'geometry': {'coordinates': [-121.86185, 37.33739],
                            'type': 'Point'},
               'id': 688518587,
               'propertie

...

                                             'months with the City of San '
                                             "Jose's Cultural Affairs Office "
                                             'and the Environmental Services '
                                             "Department. He is the City's "
                                             'first Photographer-in-Residence '
                                             'at their Wastewater Treatment '
                                             'Facility. Like ancient Rome our '
                                             'contemporary culture was made '
                                             'possible by a vast system of '
                                             'water delivery and sewage '
                                             'treatment to maintain a large '
                                             'and healthy population. Most of '
                                             'us are generally u

                                             'figure skating champions: Brian '
                                             'Boitano,\xa0Peggy '
                                             'Fleming-Jenkins, Rudy Galindo, '
                                             'Debi Thomas and\xa0Kristi '
                                             'Yamaguchi. Five Skaters honors '
                                             'the art and sport of\xa0figure '
                                             'skating, San Jose’s local '
                                             'champions and others who\xa0have '
                                             'contributed to its vitality, '
                                             'tradition and popularity.\xa0The '
                                             'artwork consists of three parts: '
                                             'five monumental\xa0columns, a '
                                             'large terrazzo ice rink, 

                            'type': 'Point'},
               'id': 3693605911,
               'properties': {'address': '45 N 1st Street',
                              'artist': 'Ricky Watts',
                              'artist_site': '',
                              'city': 'San José',
                              'country': 'United States',
                              'crossStreet': None,
                              'description': 'Mural exhibited at the 2017 <a '
                                             "href='https://www.powwowsanjose.com/' "
                                             "target='_blank'>POW! WOW! San "
                                             'José</a> in collaboration with '
                                             '<a '
                                             "href='https://www.empiresevenstudios.com' "
                                             "target='_blank'>Empire Seven "
                                             'Studios</a>.

               'errors': None,
               'geometry': {'coordinates': [-121.891267, 37.3471033],
                            'type': 'Point'},
               'id': 3971229800,
               'properties': {'address': '488 North 6th',
                              'artist': 'Peter Molica',
                              'artist_site': '',
                              'city': 'San Jose',
                              'country': None,
                              'crossStreet': None,
                              'description': 'Capiz, a series of stained-glass '
                                             'artisan windows welcome all '
                                             'visitors entering the main foyer '
                                             'of the JTC Northside Community '
                                             'Center in San Jose.\xa0 '
                                             'Kaleidoscopic and delicate, the '
                                        

                              'sourceOfInformation': None,
                              'sourceURL': None,
                              'state': 'CA',
                              'title': 'La Gran Cultura Resonance',
                              'website': None},
               'type': 'Feature'},
              {'addToMap': None,
               'errors': None,
               'geometry': {'coordinates': [-121.88019, 37.32226],
                            'type': 'Point'},
               'id': 4153382864,
               'properties': {'address': '921 S 1st St',
                              'artist': 'Maria Alguilar',
                              'artist_site': '',
                              'city': 'San José',
                              'country': 'United States',
                              'crossStreet': None,
                              'description': 'The artwork was created in 1991. '
                                             'The work is a story of two '
       

# Flatten json to fit in pandas dataframe

In [ ]:
import json 
import pandas as pd 
from pandas.io.json import json_normalize #package for flattening json in pandas df

#load json object
with open(fname) as f:
    d = json.load(f)

#lets put the data into a pandas df
#clicking on consolidated-artworks.json under "Input Files"
#tells us parent node WE WANT TO EDIT is 'features'
artwrk = json_normalize(d['features'])
artwrk.head(len(artwrk))

# edit 'properties.artist' column keeping text tag as name and putting attrs into 'properties.artist_site'

In [ ]:
artwrk_cpy.head()

# title column looks clean

In [ ]:
artwrk['properties.title']

In [ ]:
artwrk['properties.title'].iloc[41]

In [ ]:
soup = BeautifulSoup(nycphil['properties.description'].iloc[0],'lxml')

In [ ]:
soup.text


In [ ]:
nycphil['properties.description'].iloc[0]

In [ ]:
type(soup.find('a'))

In [ ]:
tags = soup.find_all('a')

In [ ]:
print(tags)
for t in tags:
    print(t.text, t.attrs['href'])

In [ ]:
soup2.absolute_links

In [ ]:
#edit 'properties.artist' column keeping text tag as name and putting attrs into 'properties.artist_site'

In [ ]:
with open(fname) as fp:
    soup2 = BeautifulSoup(fp, 'lxml')



In [ ]:
len(soup2.find_all('a'))

In [ ]:
soup2.absolute_links